In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K
# print(K.tensorflow_backend._get_available_gpus())


from scipy.special import softmax
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time

import os

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1  # exploration rate
        self.epsilon_min = 0.15
        self.epsilon_decay = 0.9995
        self.model = self._build_model()
        self.walk = deque(maxlen = 900)
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        
        model.add(Dense(100, input_dim=self.state_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(100, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        
        
        model.add(Dense(100, activation='relu'))
        model.add(BatchNormalization())
        
        model.add(Dense(self.action_size, activation='softmax'))
        
        model.compile(loss='categorical_crossentropy', optimizer=Adam())
        model.summary()
        return model
    
    def remember(self, state, action, reward, next_state, done):
#         print(state)
#         print(reward)
        self.memory.append((state, action, reward, next_state, done))
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            chose = np.random.randint(0,4)
            return chose
        
        act_values = 0
        act_values = self.model.predict(state)
        chose = np.argmax(act_values)
#         print(act_values, chose)
        if chose == 0:
            return chose
        if chose == 1:
            return chose
        if chose == 2:
            return chose
        if chose == 3:
            return chose
        return act_values
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        xs = []
        ys = []
        
        for state, action, reward, next_state, done in minibatch:
            target = reward

            if not done:
#                 if reward >= 0:
                
                target = reward + np.multiply (self.gamma , self.model.predict(next_state)[0] )
#                 else:
#                     target = np.multiply (-1 , self.model.predict(next_state)[0] )
            else:
                target = np.multiply (self.gamma , self.model.predict(next_state)[0] )
#             print(target)

            xs.append(state[0])

            ys.append(target)
#             ys = np.array(ys)
#             print(ys.shape)
        xs = np.array(xs)
        ys = np.array(ys)
        self.model.fit(xs, ys, epochs= 1, verbose=0 , batch_size=batch_size)
                
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        random.shuffle(self.memory)
#         self.memory = deque(self.memory)
        
        
        if np.random.rand() > 0.8:
            self.memory.pop()

    
    def save_model(self):
        self.model.save('./checkpoint.h5')
        np.save("game_memory", self.memory)

    def load_model(self):
        self.model.load_weights('./checkpoint.h5')
        self.memory = np.load("game_memory.npy", allow_pickle=True)
        self.memory = deque(self.memory)